# Exercise 2 - Optimizing Model Training

The benefit of cloud compute is that it offers a cost-effective way to scale out your experiment workflow and try different algorithms and parameters in order to optimize your model's performance; and that's what we'll explore in this exercise.

> **Important**: This exercise assumes you have completed the previous exercises in this series - specifically, you must have:
>
> - Created an Azure ML Workspace.
> - Uploaded the diabetes*.csv data files to the workspace's default datastore.
> - Registered a **diabetes dataset** dataset in the workspace.
>
> If you haven't done that, nobody's going to do it for you!

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK. Let's start by ensuring you still have the latest version installed.

In [ ]:
!pip install --upgrade azureml-sdk[notebooks,automl,explain]

import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Now you're ready to connect to your workspace.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## Use *Hyperdrive* to Determine Optimal Parameter Values

The remote compute you created is a four-node cluster, and you can take advantage of this to execute multiple experiment runs in parallel. One key reason to do this is to try training a model with a range of different hyperparameter values.

Azure ML includes a feature called *hyperdrive* that enables you to randomly try different values for one or more hyperparameters, and find the best performing trained model based on a metric that you specify - such as *Accuracy* or *Area Under the Curve (AUC)*.

> **More Information**: For more information about Hyperdrive, see the [Azure ML documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters).

Let's run a Hyperdrive experiment on the remote compute you have provisioned. First, we'll create the experiment and its associated folder.

In [ ]:
import os
from azureml.core import Experiment

# Create an experiment
hyperdrive_experiment_name = 'diabetes_hyperdrive'
hyperdrive_experiment = Experiment(workspace = ws, name = hyperdrive_experiment_name)

# Create a folder for the experiment files
hyperdrive_experiment_folder = './' + hyperdrive_experiment_name
os.makedirs(hyperdrive_experiment_folder, exist_ok=True)

print("Experiment:", hyperdrive_experiment.name)

Now we'll create the Python script our experiment will run in order to train a model.

In [ ]:
%%writefile $hyperdrive_experiment_folder/diabetes_training.py
# Import libraries
import argparse
import joblib
from azureml.core import Workspace, Dataset, Experiment, Run
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Set regularization parameter
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
args = parser.parse_args()
reg = args.reg_rate

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
print("Loading Data...")
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe() # Get the training data from the estimator input

# Separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/diabetes.pkl')

run.complete()

You can use the *Hyperdrive* feature of Azure ML to run multiple experiments in parallel, using different values for the **regularization** parameter to find the optimal value for the data. To help  scale this out, you'll run the Hyperdrive experiment on one of the training compute clusters you created in the previous exercise.

> **Note**: If you didn't create it in the previous exercise, don't worry - the code below will create it

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster
cluster_name = "aml-compute1"

# Get the existing cluster
try:
    compute1 = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create it if it doesn't exist
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=4)
    compute1 = ComputeTarget.create(ws, cluster_name, compute_config)

compute1.wait_for_completion(show_output=True)

Now you're ready to run the experiment.

This will take some time to run. First Azure ML must initialize the compute cluster and prepare the Python environment. Then, the experiment will launch a child run for each hyperparameter value combination to be evaluated.

In [ ]:
from azureml.train.hyperdrive import GridParameterSampling, BanditPolicy, HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import choice
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn # We'll use the SKLearn estimator

# Sample a range of parameter values
params = GridParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--regularization': choice(0.001, 0.005, 0.01, 0.05, 0.1, 1.0)
    }
)

# Set evaluation policy to stop poorly performing training runs early
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")

# Create an estimator that uses the remote compute
hyper_estimator = SKLearn(source_directory=hyperdrive_experiment_folder,
                           inputs=[diabetes_ds.as_named_input('diabetes')], # Pass the dataset as an input
                           compute_target = compute1,
                           conda_packages=['pandas','ipykernel','matplotlib'], #The estimator already includes scikit-learn
                           pip_packages=['azureml-sdk','argparse','pyarrow'],
                           entry_script='diabetes_training.py')

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(estimator=hyper_estimator, 
                          hyperparameter_sampling=params, 
                          policy=policy, 
                          primary_metric_name='AUC', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=6,
                          max_concurrent_runs=4)


# Run the experiment
hyperdrive_run = hyperdrive_experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
RunDetails(hyperdrive_run).show()

You can view the experiment run status in the widget above. You can also view the main Hyperdrive experiment run and its child runs in the [Azure ML Studio web interface](https://ml.azure.com).

> **Tip**: While the experiment is running, skip ahead to the **Use *Automated ML* to Find the Best Model for your Data** task below. Then, after you've started the Auto ML experiment, you can come back here and wait for the status of the hyperdrive experiment in the widget above to be **Completed**.

You can find the best model based on the performance metric you specified (in this case, the one with the best AUC).

In [ ]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdrive_run_metrics = best_hyperdrive_run.get_metrics()
hyperdrive_parameter_values = best_hyperdrive_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_hyperdrive_run.id)
print(' -AUC:', best_hyperdrive_run_metrics['AUC'])
print(' -Accuracy:', best_hyperdrive_run_metrics['Accuracy'])
print(' -Regularization Rate:',hyperdrive_parameter_values)

Since we've found the best run, we can register the model it trained.

In [ ]:
from azureml.core import Model

# Register model
best_hyperdrive_run.register_model(model_path='outputs/diabetes.pkl', model_name='diabetes', tags={'Training context':'Hyperdrive'}, properties={'AUC': best_hyperdrive_run_metrics['AUC'], 'Accuracy': best_hyperdrive_run_metrics['Accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **Tip**: If you previously skipped ahead to start the Auo ML experiment, scroll down and view its widget to await its completion.

## Use *Automated ML* to Find the Best Model for your Data

Hyperparameter tuning has helped us find the optimal regularization rate for our logistic regression model, but we might get better results by trying a different algorithm, and by performing some basic feature-engineering, such as scaling numeric feature values. You could just create lots of different training scripts that apply various scikit-learn algorithms, and try them all until you find the best result; but Azure ML provides a feature called *Automated Machine Learning* (or *Auto ML*) that can do this for you.

> **Note**: You can use the Auto ML wizard in the [Azure ML Studio web interface](https://ml.azure.com) to submit an Auto ML experiment, or you can initiate Auto ML using the Azure ML SDK. The SDK gives you greater control over the settings for the Auto ML experiment, but the visual interface is easier to use.

You don't need to create a training script (Auto ML will do that for you), but you do need to load the training data.

There are a few ways you can do this, but one of the easiest is to use a dataset. In this case, we'll use the diabetes dataset, but we'll also split it into training and test datasets.

In [ ]:
# Get the training dataset
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Just as before, you can scale out the experiment by running it on a training compute cluster, so let's get a reference to one.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your cluster
cluster_name = "aml-compute2"

# Get the existing cluster
try:
    compute2 = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create it if it doesn't exist
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2', max_nodes=4)
    compute2 = ComputeTarget.create(ws, cluster_name, compute_config)

compute2.wait_for_completion(show_output=True)

Now you're ready to configure the Auto ML experiment. To do this, you'll need a run configuration that includes the required packages for the experiment environment, and a set of configuration settings that tells Auto ML how many options to try, which metric to use when evaluating models, and so on.

> **More Information**: For more information about options when using Auto ML, see the [Azure ML documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train).

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.train.automl import AutoMLConfig
import time
import logging


automl_run_config = RunConfiguration(framework="python")
automl_run_config.environment.docker.enabled = True

automl_settings = {
    "name": "Diabetes_AutoML_{0}".format(time.time()),
    "iteration_timeout_minutes": 10,
    "iterations": 6,
    "primary_metric": 'AUC_weighted',
    "preprocess": False,
    "max_concurrent_iterations": 4
}

automl_config = AutoMLConfig(task='classification',
                             debug_log='automl_errors.log',
                             compute_target=compute2,
                             run_configuration=automl_run_config,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             model_explainability=True,
                             **automl_settings,
                             )

print("Ready for Auto ML run.")

OK, we're ready to go. Let's start the Auto ML run.

This will take a significant amount of time. Eventually, a widget showing the run details will be displayed. You can monitor the compute and experiment status in the [Azure ML Studio web interface](https://ml.azure.com).

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()

> **Tip**: If you skipped ahead to start the Auto ML experiment while the Hyperdrive experiment is running, go back now and await the Hyperdrive experiment's completion.

View the output of the experiment in the widget; and when all of the runs have finished, click the run that produced the best result to see its details.

You can also view the AutoML experiment run and the models it trained in the [Azure ML Studio web interface](https://ml.azure.com). On the **Automated ML** page, click the **Run ID** of the latest experiment run, and then on the **Models** tab you can view the details for each model.

Let's get the best run and the model that was generated (you can ignore any warnings about Azure ML package versions that might appear).

In [ ]:
best_automl_run, fitted_model = automl_run.get_output()
print(best_automl_run)
print(fitted_model)
best_automl_run_metrics = best_automl_run.get_metrics()
for metric_name in best_automl_run_metrics:
    metric = best_automl_run_metrics[metric_name]
    print(metric_name, metric)

One of the options you used was to include model *explainability*. This uses a test dataset to evaluate the importance of each feature. You can retrieve this information from the run.

> **Note**: The model explanation is generated by a separate experiment run that may take some additional time to finish. If running the cell below results in an error, wait a few minutes and try again!

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

client = ExplanationClient.from_run(best_automl_run)
engineered_explanations = client.download_model_explanation(raw=True)
feature_importances = engineered_explanations.get_feature_importance_dict()

# Overall feature importance (the Feature value is the column index in the training data)
print("Feature\tImportance")
for key, value in feature_importances.items():
    print(key, "\t", value)

Finally, having found the best performing model, you can register it.

In [ ]:
from azureml.core import Model

# Register model
best_automl_run.register_model(model_path='outputs/model.pkl', model_name='diabetes', tags={'Training context':'Auto ML'}, properties={'AUC': best_automl_run_metrics['AUC_weighted'], 'Accuracy': best_automl_run_metrics['accuracy']})

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

Now you've seen several ways to leverage the high-scale compute capabilities of the cloud to experiment with model training and find the best performing model for your data